# Anime User Based

In [22]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [6]:
anime = pd.read_csv('../anime-recommendations-database/anime.csv')
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


anime_id = ID identifying an anime
genre = list of genres for this anime
rating = average rating out of 10 for this anime

In [7]:
animeRating = pd.read_csv('../anime-recommendations-database/rating.csv')
animeRating.tail()

,user_id,anime_id,rating
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9
7813736,73516,8074,9


anime_id = anime the user has rated
rating = rating out of 10 (-1 if user watched but didn't assign a rating)

### Cleaning Data for Missing Values

In [8]:
#Checking which values are null
print anime.isnull().sum()
print animeRating.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64
user_id     0
anime_id    0
rating      0
dtype: int64


In [9]:
anime['genre'] = anime['genre'].fillna('None')
anime['type'] = anime['type'].fillna('None')
anime['rating'] = anime['rating'].fillna('None')
anime.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [10]:
animeRating = animeRating[animeRating.rating > 0]
animeRating.rating.unique()

array([10,  8,  6,  9,  7,  3,  5,  4,  1,  2], dtype=int64)

In [38]:
fullMergedAnime = animeRating.merge(anime, left_on = 'anime_id', right_on = 'anime_id', suffixes= ['_user', ''])
fullMergedAnime.head()

,user_id,anime_id,rating_user,name,genre,type,episodes,rating,members
0,1,8074,10,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
1,3,8074,6,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
2,5,8074,2,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
3,12,8074,6,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
4,14,8074,6,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892


### Splitting and Normalizing Data

In [35]:
mergedSubAnime = fullMergedAnime[['user_id', 'anime_id', 'name', 'rating_user']]

#Limiting the dataframe
mergedSubAnime = mergedSubAnime[mergedSubAnime.user_id <= 6000]

In [36]:
trainData, testData = train_test_split(mergedSubAnime, test_size=0.2)

In [37]:
piv = trainData.pivot_table(index=['user_id'], columns=['name'], values='rating_user')
print(piv.shape)

(5592, 6803)


In [33]:
piv.head()

name,&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,&quot;Bungaku Shoujo&quot; Memoire,&quot;Bungaku Shoujo&quot; Movie,.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,.hack//Gift,.hack//Intermezzo,.hack//Liminality,.hack//Quantum,...,gdgd Fairies 2,iDOLM@STER Xenoglossia,iDOLM@STER Xenoglossia Specials,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,◯
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
piv_norm = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
piv_norm.fillna(0, inplace=True)

In [13]:
piv_norm = piv_norm.T

In [14]:
piv_norm = piv_norm.loc[:, (piv_norm != 0).any(axis=0)]

In [16]:
piv_sparse = sp.sparse.csr_matrix(piv_norm.values)

### User Cosine Similarity

In [17]:
userCosineSimilarity = cosine_similarity(piv_sparse.T)
print userCosineSimilarity

[[1.         0.06399734 0.17047619 ... 0.04277882 0.03609253 0.11376404]
 [0.06399734 1.         0.06692376 ... 0.00826342 0.0110427  0.14576958]
 [0.17047619 0.06692376 1.         ... 0.04092356 0.0902815  0.08578008]
 ...
 [0.04277882 0.00826342 0.04092356 ... 1.         0.08711626 0.08930886]
 [0.03609253 0.0110427  0.0902815  ... 0.08711626 1.         0.04099017]
 [0.11376404 0.14576958 0.08578008 ... 0.08930886 0.04099017 1.        ]]


In [18]:
userSimData = pd.DataFrame(userCosineSimilarity, index = piv_norm.columns, columns = piv_norm.columns)

In [19]:
userSimData.head()

user_id,3,5,7,8,10,11,12,14,16,17,...,4990,4991,4992,4993,4995,4996,4997,4998,4999,5000
user_id,,,,,,,,,,,,,,,,,,,,,
3,1.000000,0.063997,0.170476,0.022019,0.110446,7.544810e-02,0.094138,-0.000878,0.047537,0.195305,...,0.015445,0.033642,0.033998,0.035527,-0.042146,0.080141,0.033314,0.042779,0.036093,0.113764
5,0.063997,1.000000,0.066924,0.012951,0.026877,3.555051e-02,0.074414,0.111004,-0.002012,0.120638,...,0.007609,0.038158,0.028000,0.076681,0.004772,-0.003391,0.034073,0.008263,0.011043,0.145770
7,0.170476,0.066924,1.000000,-0.012122,0.015496,-1.624052e-02,0.055242,0.095136,0.019608,0.131156,...,-0.001424,0.031665,0.043799,-0.019429,-0.004971,0.051204,-0.027792,0.040924,0.090282,0.085780
8,0.022019,0.012951,-0.012122,1.000000,-0.052705,1.215475e-02,0.004495,-0.051039,0.000000,0.001142,...,0.000000,0.000000,0.000000,-0.005206,0.000000,0.000000,0.000000,0.004398,0.000000,0.013231
10,0.110446,0.026877,0.015496,-0.052705,1.000000,-1.040834e-16,0.014213,0.043936,0.000000,0.061266,...,0.000000,0.000000,0.000000,0.070297,0.026127,0.000000,0.000000,0.000000,0.000000,0.044732


In [20]:
def top_Similar_Users(currentUser):
    if currentUser not in piv_norm.columns:
        return("No data available for {}".format(currentUser))
    
    similarUsers = userSimData.sort_values(by=currentUser, ascending=False).index[1:20]
    similarUsersResults = userSimData.sort_values(by=currentUser, ascending=False).loc[:,currentUser].tolist()[1:20]
    similarUsersZipped = zip(similarUsers, similarUsersResults)
#     print similarUsersZipped
    
    return similarUsersZipped

In [21]:
def avgRatingsUser(user):
    userRatings = piv.loc[user, :]
    avgUserRatingList = []
    for rating in userRatings:
        if not np.isnan(rating):
            avgUserRatingList.append(rating)
        else:
            continue
    return sum(avgUserRatingList)/len(avgUserRatingList)

In [22]:
# Define for current existing user that has some data available

def ratingsUserAnime(currentUser, animeName, similarUsers):
#     ratings = {}
#     currentUserAnime = piv.loc[currentUser, :]
#     print currentUserAnime
    ratingWeight = []
    similarityWeight = []
    avgCurrentUserRating = avgRatingsUser(currentUser)
    
    if currentUser in piv_norm.columns:
        for user, userSimilarity in similarUsers:
            #Average of the similar user ratings
            avgUserRating = avgRatingsUser(user)
            
            #Find the rated anime
            userAnimeRating = piv.loc[user, animeName]
#             print userAnimeRating
            if np.isnan(userAnimeRating):
                continue
            elif not np.isnan(userAnimeRating):
#                 print userAnimeRating
                ratingWeight.append((userAnimeRating - avgUserRating)*userSimilarity)
                similarityWeight.append(abs(userSimilarity))
    else:
        return ("No data available for: {}".format(currentUser))
    
    if sum(similarityWeight) > 0:
        return avgCurrentUserRating + (sum(ratingWeight)/sum(similarityWeight))
    else:
        return avgCurrentUserRating

In [23]:
similarUsers = top_Similar_Users(7)
print ratingsUserAnime(10, "Zombie-Loan", similarUsers)

8.768115942028986


In [24]:
def predictUserRatings(currentUser):
    ratingsNone = {}
    animeColumn = list(piv.columns.values)
    currentUserRatings = piv.loc[currentUser, :].tolist()
    currentUserAnime = dict(zip(animeColumn, currentUserRatings))
    similarUsers = top_Similar_Users(currentUser)
#     print len(currentUserAnime)
    
    for animeName, animeRating in currentUserAnime.items():
        if np.isnan(animeRating):
            ratingsNone[animeName] = ratingsUserAnime(currentUser, animeName, similarUsers)
            print ("User: {}, Rating: {}".format(user, ratingsNone[user]))
    
    return ratingsNone
        

In [25]:
# predictUserRatings(7)

### User Pearson Similarity

In [26]:
piv_user = piv_norm.values
userCovariance = np.cov(piv_user.T)

In [27]:
userPearsonSimilarity = np.corrcoef(userCovariance)
# print testing2

In [28]:
userPearsonSimData = pd.DataFrame(userPearsonSimilarity, index = piv_norm.columns, columns = piv_norm.columns)

In [29]:
userPearsonSimData.head()

user_id,3,5,7,8,10,11,12,14,16,17,...,4990,4991,4992,4993,4995,4996,4997,4998,4999,5000
user_id,,,,,,,,,,,,,,,,,,,,,
3,1.000000,0.390599,0.540247,0.131393,0.230468,0.338315,0.442852,0.349422,0.321334,0.624789,...,0.116088,0.260055,0.225033,0.341106,-0.137320,0.393180,0.087866,0.346094,0.247075,0.571032
5,0.390599,1.000000,0.456861,0.003083,0.264419,0.175298,0.321363,0.551465,0.165403,0.601768,...,0.158756,0.292935,0.235149,0.498507,0.027153,0.121315,0.183013,0.366469,0.259727,0.615792
7,0.540247,0.456861,1.000000,0.045588,0.146607,0.105991,0.374586,0.543455,0.315055,0.661314,...,0.160566,0.263536,0.259735,0.366285,-0.049507,0.292652,-0.016443,0.459601,0.437305,0.638354
8,0.131393,0.003083,0.045588,1.000000,-0.161154,0.059567,-0.006389,-0.029386,0.026510,0.016668,...,-0.044372,-0.007285,0.001524,-0.020292,0.002354,0.010841,-0.022189,0.034068,-0.010836,0.096772
10,0.230468,0.264419,0.146607,-0.161154,1.000000,0.049037,0.224339,0.223979,0.081027,0.313504,...,0.068301,0.191987,0.059808,0.359261,0.037801,0.113908,0.107205,0.159429,0.140987,0.232634


In [30]:
def top_Similar_Pearson_Users(currentUser):
    if currentUser not in piv_norm.columns:
        return("No data available for {}".format(currentUser))
    
    similarUsers = userPearsonSimData.sort_values(by=currentUser, ascending=False).index[1:20]
    similarUsersResults = userPearsonSimData.sort_values(by=currentUser, ascending=False).loc[:,currentUser].tolist()[1:20]
    similarUsersZipped = zip(similarUsers, similarUsersResults)
#     print similarUsersZipped
    
    return similarUsersZipped

In [31]:
similarPearsonUsers = top_Similar_Pearson_Users(7)
print ratingsUserAnime(7, "Zombie-Loan", similarPearsonUsers)

6.399927865841151


In [32]:
def predictPearsonUserRatings(currentUser):
    ratingsNone = {}
    animeColumn = list(piv.columns.values)
    currentUserRatings = piv.loc[currentUser, :].tolist()
    currentUserAnime = dict(zip(animeColumn, currentUserRatings))
    similarUsers = top_Similar_Pearson_Users(currentUser)
#     print len(currentUserAnime)
    
    for animeName, animeRating in currentUserAnime.items():
        if np.isnan(animeRating):
            ratingsNone[animeName] = ratingsUserAnime(currentUser, animeName, similarUsers)
            print ("User: {}, Rating: {}".format(user, ratingsNone[user]))
    
    return ratingsNone

### Item Cosine Similarity

In [33]:
itemCosineSimilarity = cosine_similarity(piv_sparse)

In [34]:
itemSimData = pd.DataFrame(itemCosineSimilarity, index = piv_norm.index, columns = piv_norm.index)

In [35]:
itemSimData.head()

name,&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,&quot;Bungaku Shoujo&quot; Memoire,&quot;Bungaku Shoujo&quot; Movie,.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,.hack//Gift,.hack//Intermezzo,.hack//Liminality,.hack//Quantum,...,gdgd Fairies 2,iDOLM@STER Xenoglossia,iDOLM@STER Xenoglossia Specials,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,◯
name,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,1.000000,0.172085,0.186285,0.020928,0.061367,0.032293,0.004453,-0.043450,-0.017462,0.017125,...,0.0,-0.005201,0.023101,0.000000,0.075029,0.106081,0.133147,0.151144,0.079971,0.0
&quot;Bungaku Shoujo&quot; Memoire,0.172085,1.000000,0.388434,-0.020467,0.037005,-0.059632,-0.007363,-0.052221,-0.033056,0.036570,...,0.0,0.004336,-0.042659,-0.048821,-0.041747,-0.050302,-0.045607,-0.016375,-0.091434,0.0
&quot;Bungaku Shoujo&quot; Movie,0.186285,0.388434,1.000000,-0.048346,0.044058,-0.027781,-0.023449,-0.107170,-0.048258,0.030136,...,0.0,0.002020,-0.019874,-0.007470,-0.037358,-0.038744,-0.018042,0.028878,0.000960,0.0
.hack//G.U. Returner,0.020928,-0.020467,-0.048346,1.000000,0.559906,-0.149000,0.371347,0.633474,0.236892,0.199659,...,0.0,-0.002475,0.026172,0.000000,-0.052349,-0.068782,-0.076498,-0.104512,-0.141105,0.0
.hack//G.U. Trilogy,0.061367,0.037005,0.044058,0.559906,1.000000,-0.153814,0.194511,0.290773,0.062776,0.073942,...,0.0,-0.001661,0.017569,-0.017556,-0.027834,-0.009529,0.008717,0.005253,-0.049467,0.0


In [36]:
def top_Similar_Animes(anime):
    similarAnime = itemSimData.sort_values(by=anime, ascending=False).index[1:20]
    similarAnimeResults = itemSimData.sort_values(by=anime, ascending=False).loc[:,anime].tolist()[1:20]
    similarAnimeZipped = zip(similarAnime, similarAnimeResults)
#     print similarUsersZipped
    
    return similarAnimeZipped

In [37]:
def avgRatingItem(item):
    itemRatings = piv.loc[:, item]
    avgItemRatingList = []
    for rating in itemRatings:
        if not np.isnan(rating):
            avgItemRatingList.append(rating)
        else:
            continue
    return sum(avgItemRatingList)/len(avgItemRatingList)

In [38]:
def ratingsItemAnime(currentUser, animeName, similarItems):
    ratingWeight = []
    similarityWeight = []
    avgCurrentItemRating = avgRatingItem(animeName)
    
    if currentUser in piv_norm.columns:
        for item, itemSimilarity in similarItems:
            #Average of the similar item ratings
            avgItemRating = avgRatingItem(item)
            
            #Find the rated anime
            itemAnimeRating = piv.loc[currentUser, item]
            if np.isnan(itemAnimeRating):
                continue
            elif not np.isnan(itemAnimeRating):
                ratingWeight.append((itemAnimeRating - avgItemRating)*itemSimilarity)
                similarityWeight.append(abs(itemSimilarity))
    else:
        return ("No data available for: {}".format(currentUser))
    
    if sum(similarityWeight) > 0:
        return avgCurrentItemRating + (sum(ratingWeight)/sum(similarityWeight))
    else:
        return avgCurrentItemRating

In [39]:
similarAnimes = top_Similar_Animes("Zombie-Loan")
print ratingsItemAnime(7, "Zombie-Loan", similarAnimes)

6.392857142857143


In [40]:
def predictItemRatings(currentItem):
    ratingsNone = {}
    animeRow = list(piv.index.values)
    currentItemRatings = piv.loc[:, currentItem].tolist()  
    currentUserAnime = dict(zip(animeRow, currentItemRatings))
    similarAnimes = top_Similar_Animes(currentItem)
#     print len(currentUserAnime)
#     similarUsers = top_Similar_Users(currentUser)
#     print currentUserAnime
    
    for user, animeRating in currentUserAnime.items():
        if np.isnan(animeRating):
            ratingsNone[user] = ratingsItemAnime(user, currentItem, similarAnimes)
#             print ("User: {}, Rating: {}".format(user, ratingsNone[user]))
    return ratingsNone
    

In [ ]:
# predictItemRatings("Zombie-Loan")

{1: 'No data available for: 1',
 2: 'No data available for: 2',
 5: 7.192857142857143,
 7: 6.392857142857143,
 8: 7.192857142857143,
 9: 'No data available for: 9',
 10: 7.192857142857143,
 11: 7.192857142857143,
 12: 7.192857142857143,
 14: 7.192857142857143,
 16: 7.192857142857143,
 17: 7.192857142857143,
 18: 7.192857142857143,
 19: 7.192857142857143,
 20: 7.192857142857143,
 21: 7.192857142857143,
 22: 'No data available for: 22',
 23: 7.192857142857143,
 24: 7.192857142857143,
 25: 7.192857142857143,
 26: 7.192857142857143,
 27: 7.192857142857143,
 28: 7.192857142857143,
 29: 7.192857142857143,
 30: 7.192857142857143,
 31: 7.192857142857143,
 32: 7.192857142857143,
 33: 7.192857142857143,
 34: 7.192857142857143,
 35: 'No data available for: 35',
 36: 7.192857142857143,
 37: 7.192857142857143,
 38: 3.4201298701298706,
 39: 7.192857142857143,
 40: 7.192857142857143,
 41: 7.192857142857143,
 42: 7.192857142857143,
 43: 6.420129870129871,
 44: 7.192857142857143,
 45: 7.192857142857143

### Item Pearson Similarity

In [ ]:
piv_item = piv_norm.values
itemCovariance = np.cov(piv_item)

In [ ]:
itemPearsonSimilarity = np.corrcoef(itemCovariance)

In [ ]:
itemPearsonSimData = pd.DataFrame(itemPearsonSimilarity, index = piv_norm.index, columns = piv_norm.index)

In [ ]:
itemPearsonSimData.head()

In [ ]:
def top_Pearson_Similar_Animes(anime):
    similarAnime = itemPearsonSimData.sort_values(by=anime, ascending=False).index[1:20]
    similarAnimeResults = itemPearsonSimData.sort_values(by=anime, ascending=False).loc[:,anime].tolist()[1:20]
    similarAnimeZipped = zip(similarAnime, similarAnimeResults)
#     print similarUsersZipped
    
    return similarAnimeZipped

In [ ]:
def predictPearsonItemRatings(currentItem):
    ratingsNone = {}
    animeRow = list(piv.index.values)
    currentItemRatings = piv.loc[:, currentItem].tolist()  
    currentUserAnime = dict(zip(animeRow, currentItemRatings))
    similarAnimes = top_Pearson_Similar_Animes(currentItem)
#     print len(currentUserAnime)
#     similarUsers = top_Similar_Users(currentUser)
#     print currentUserAnime
    
    for user, animeRating in currentUserAnime.items():
        if np.isnan(animeRating):
            ratingsNone[user] = ratingsItemAnime(user, currentItem, similarAnimes)
#             print ("User: {}, Rating: {}".format(user, ratingsNone[user]))
    return ratingsNone
    

In [ ]:
predictPearsonItemRatings("Zombie-Loan")

### Results